In [ ]:
import numpy as np
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
voting_data=pd.read_csv('/content/drive/MyDrive/CP project/Project/Raw Datasets/1976-2020-president.csv')
voting_data.head()

,year,state,state_po,state_fips,state_cen,state_ic,office,candidate,party_detailed,writein,candidatevotes,totalvotes,version,notes,party_simplified
0,1976,ALABAMA,AL,1,63,41,US PRESIDENT,"CARTER, JIMMY",DEMOCRAT,False,659170,1182850,20210113,NaN,DEMOCRAT
1,1976,ALABAMA,AL,1,63,41,US PRESIDENT,"FORD, GERALD",REPUBLICAN,False,504070,1182850,20210113,NaN,REPUBLICAN
2,1976,ALABAMA,AL,1,63,41,US PRESIDENT,"MADDOX, LESTER",AMERICAN INDEPENDENT PARTY,False,9198,1182850,20210113,NaN,OTHER
3,1976,ALABAMA,AL,1,63,41,US PRESIDENT,"BUBAR, BENJAMIN """"BEN""""",PROHIBITION,False,6669,1182850,20210113,NaN,OTHER
4,1976,ALABAMA,AL,1,63,41,US PRESIDENT,"HALL, GUS",COMMUNIST PARTY USE,False,1954,1182850,20210113,NaN,OTHER


In [ ]:
voting_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4287 entries, 0 to 4286
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   year              4287 non-null   int64  
 1   state             4287 non-null   object 
 2   state_po          4287 non-null   object 
 3   state_fips        4287 non-null   int64  
 4   state_cen         4287 non-null   int64  
 5   state_ic          4287 non-null   int64  
 6   office            4287 non-null   object 
 7   candidate         4000 non-null   object 
 8   party_detailed    3831 non-null   object 
 9   writein           4284 non-null   object 
 10  candidatevotes    4287 non-null   int64  
 11  totalvotes        4287 non-null   int64  
 12  version           4287 non-null   int64  
 13  notes             0 non-null      float64
 14  party_simplified  4287 non-null   object 
dtypes: float64(1), int64(7), object(7)
memory usage: 502.5+ KB


In [ ]:
voting_data.columns

Index(['year', 'state', 'state_po', 'state_fips', 'state_cen', 'state_ic',
       'office', 'candidate', 'party_detailed', 'writein', 'candidatevotes',
       'totalvotes', 'version', 'notes', 'party_simplified'],
      dtype='object')

In [ ]:
voting_data['office'].unique()

array(['US PRESIDENT'], dtype=object)

In [ ]:
voting_data.drop(['office','state_fips','state_cen','state_ic',
                  'party_detailed','writein','version','notes'],
                 axis=1,inplace=True)

voting_data.columns=['Year','State','State Initials','Candidate','Candidate votes',
                     'Total votes',"Candidate's party"]

In [ ]:
voting_data.isnull().sum()

,0
Year,0
State,0
State Initials,0
Candidate,287
Candidate votes,0
Total votes,0
Candidate's party,0


In [ ]:
candidate_na_values=voting_data[voting_data['Candidate'].isna()]
candidate_na_values["Candidate's party"].unique()

array(['OTHER', 'LIBERTARIAN', 'DEMOCRAT'], dtype=object)

In [ ]:
votes_party_per_state = voting_data.pivot_table(
    index=['Year','State Initials','State'],
    columns="Candidate's party",
    values='Candidate votes',
    aggfunc='sum',
    fill_value=0
)
votes_party_per_state.columns=['Democratic','Libertarian','Other','Republican']
votes_party_per_state


Democratic  Libertarian   Other  Republican
Year State Initials State                                                     
1976 AK             ALASKA              44058         6785    1176       71555
     AL             ALABAMA            659170         1481   18129      504070
     AR             ARKANSAS           498604            0    1028      267903
     AZ             ARIZONA            295602         7647   20828      418642
     CA             CALIFORNIA        3742284            0  179242     3882244
...                                       ...          ...     ...         ...
2020 VT             VERMONT            242820         3608   11836      112704
     WA             WASHINGTON        2369612        80500   52868     1584651
     WI             WISCONSIN         1630866            0   56991     1610184
     WV             WEST VIRGINIA      235984        10687    2599      545382
     WY             WYOMING             73491         5768    5685      193559

[612 rows x 4 columns]

In [ ]:

regions = {
    'Northeast': ['CONNECTICUT', 'MAINE', 'MASSACHUSETTS', 'NEW HAMPSHIRE', 'NEW JERSEY',
                  'NEW YORK', 'PENNSYLVANIA', 'RHODE ISLAND', 'VERMONT'],
    'Midwest': ['ILLINOIS', 'INDIANA', 'IOWA', 'KANSAS', 'MICHIGAN', 'MINNESOTA', 'MISSOURI',
                'NEBRASKA', 'NORTH DAKOTA', 'OHIO', 'SOUTH DAKOTA', 'WISCONSIN'],
    'South': ['ALABAMA', 'ARKANSAS', 'DELAWARE', 'FLORIDA', 'GEORGIA', 'KENTUCKY', 'LOUISIANA',
              'MARYLAND', 'MISSISSIPPI', 'NORTH CAROLINA', 'SOUTH CAROLINA', 'TENNESSEE',
              'TEXAS', 'VIRGINIA', 'WEST VIRGINIA','OKLAHOMA', 'DISTRICT OF COLUMBIA'],
    'West': ['ALASKA', 'ARIZONA', 'CALIFORNIA', 'COLORADO', 'HAWAII', 'IDAHO', 'MONTANA',
             'NEVADA', 'NEW MEXICO', 'OREGON', 'UTAH', 'WASHINGTON', 'WYOMING']
}

# Step 2: Add a 'Region' column to the dataframe
def assign_region(state):
    for region, states in regions.items():
        if state in states:
            return region


votes_party_per_state = votes_party_per_state.reset_index()  # Reset index to access 'state'
votes_party_per_state['Region'] = votes_party_per_state['State'].apply(assign_region)
votes_party_per_state = votes_party_per_state.set_index(['Year', 'State']).sort_index()
votes_party_per_state.head()

State Initials  Democratic  Libertarian   Other  Republican  \
Year State                                                                    
1976 ALABAMA                AL      659170         1481   18129      504070   
     ALASKA                 AK       44058         6785    1176       71555   
     ARIZONA                AZ      295602         7647   20828      418642   
     ARKANSAS               AR      498604            0    1028      267903   
     CALIFORNIA             CA     3742284            0  179242     3882244   

                Region  
Year State              
1976 ALABAMA     South  
     ALASKA       West  
     ARIZONA      West  
     ARKANSAS    South  
     CALIFORNIA   West

In [ ]:
votes_party_per_state['Winner'] = votes_party_per_state[['Democratic', 'Libertarian', 'Other', 'Republican']].idxmax(axis=1)
votes_party_per_state


State Initials  Democratic  Libertarian   Other  \
Year State                                                           
1976 ALABAMA                   AL      659170         1481   18129   
     ALASKA                    AK       44058         6785    1176   
     ARIZONA                   AZ      295602         7647   20828   
     ARKANSAS                  AR      498604            0    1028   
     CALIFORNIA                CA     3742284            0  179242   
...                           ...         ...          ...     ...   
2020 VIRGINIA                  VA     2413568        64761   19765   
     WASHINGTON                WA     2369612        80500   52868   
     WEST VIRGINIA             WV      235984        10687    2599   
     WISCONSIN                 WI     1630866            0   56991   
     WYOMING                   WY       73491         5768    5685   

                    Republican   Region      Winner  
Year State                                           
1976 ALABAMA            504070    South  Democratic  
     ALASKA              71555     West  Republican  
     ARIZONA            418642     West  Republican  
     ARKANSAS           267903    South  Democratic  
     CALIFORNIA        3882244     West  Republican  
...                        ...      ...         ...  
2020 VIRGINIA          1962430    South  Democratic  
     WASHINGTON        1584651     West  Democratic  
     WEST VIRGINIA      545382    South  Republican  
     WISCONSIN         1610184  Midwest  Democratic  
     WYOMING            193559     West  Republican  

[612 rows x 7 columns]

In [ ]:
votes_party_per_state[votes_party_per_state['Region'].isna()]

,,State Initials,Democratic,Libertarian,Other,Republican,Region,Winner
Year,State,,,,,,,


In [ ]:
votes_party_per_state.to_csv('/content/drive/MyDrive/CP project/Project/Cleaned Datasets/Votes Party per state 1976-2020.csv',index=True)


In [ ]:
# Let's compute the total votes that each party obtained during the election
votes_by_party = voting_data.pivot_table(
    index='Year',
    columns="Candidate's party",
    values='Candidate votes',
    aggfunc='sum',
    fill_value=0
)
votes_by_party.columns=['Democratic','Libertarian','Other','Republican']
votes_by_party

,Democratic,Libertarian,Other,Republican
Year,,,,
1976,40680446,95626,1954379,38870893
1980,35480948,867401,6505863,43642639
1984,37449813,227204,811015,54166829
1988,41716679,409708,817798,48642640
1992,44856747,280848,20663272,38798913
1996,47295351,465351,9625419,39003697
2000,50830580,380405,4071625,50311372
2004,58894561,369308,1212870,61872711
2008,69338846,510456,1956116,59613835


In [ ]:
votes_by_party.to_csv('/content/drive/MyDrive/CP project/Project/Cleaned Datasets/Votes by party 1976-2020.csv',index=True)
